In [1]:
import import_ipynb
import numpy as np
import math as mt
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import DistanceMap as dis

importing Jupyter notebook from DistanceMap.ipynb


In [3]:
SH_atHW21 = pd.read_csv('Data/RS_forcing_b3.0.csv', header=0, parse_dates=[0], index_col=0)

In [ ]:
## Channel geometry
Bw = 50.0 # Bottom width (unit: m)
S0 = 0.00057 # Channel bottom slope (Channel length 24 km, Elevation diff: 0.013716)
z = 100.0 # Channel side slope (unit: m)
Geometry = 2.0

## Manning's parameters
n = 0.01 # Manning coefficient (unit: -)
g = 9.8 # gravitational acceleration (unit: m/s2)
Chan_grid_size = dis.resolution # Channel grid size (unit: m)

In [ ]:
simulation_period = 9999 ## To cover the entire simulation length in hour 
SH_inlet = np.zeros((len(dis.Chan_indi), simulation_period))
SH_outlet = np.zeros((len(dis.Chan_indi), simulation_period))
SH_profile = np.zeros(((int(sum(dis.Cat_grid_num))), simulation_period)) + SH_atHW21['River_stage'][1]
Streamflow_inlet = np.zeros((len(dis.Chan_indi), simulation_period))
Streamflow_outlet = np.zeros((len(dis.Chan_indi), simulation_period)) 
Streamflow_profile = np.zeros(((int(sum(dis.Cat_grid_num))), simulation_period))
Flow_area = np.zeros((len(dis.Chan_indi), simulation_period)) 
Wr = np.zeros((len(dis.Chan_indi), simulation_period)) 

In [ ]:
SW_nt = 360

def LRMC(Qlat, SH_upstream, Chan_grid_num, Cat_num, hour, SW_tmax):
    
    ## SW_tmax: 3,600 seconds
    str_dis = np.zeros((Chan_grid_num+1, SW_nt+1)) ## str_dis: stream discharge
    dx = Chan_grid_size 
    
    ## Manning's formulation
    Area, Rad, mean_velocity = Manning(SH_upstream)
    
    ## Incremental travel time calculation
    Chan_totalLength = 0
    for i in range(0, Cat_num+1):
        Chan_totalLength += int(dis.Chan_grid_num[i]) 
    
    ## SW_tmax: 3,600 seconds (1 hour)
    travel_time = mt.ceil(((Chan_totalLength * Chan_grid_size) / mean_velocity) / SW_tmax) # travel_time: hr

    ## Ck the flood wave celerity of small waves in non-rectangular channels
    Ck = pow(g * SH_upstream, 0.5)
    
    Fr_num = mean_velocity / Ck
    
    Flow_area[Cat_num][hour] = Area
#     Wr[Cat_num][hour] = Bw + 2 * pow((pow(SH_upstream, 2) + pow(z, 2)), 0.5)
    Wr[Cat_num][hour] = Bw + 2 * SH_upstream

#     r = (1 - Rad * ((Wr[Cat_num, hour] - Wr[Cat_num, hour-1]) /\
#                  (Flow_area[Cat_num][hour] - Flow_area[Cat_num][hour-1])))
    r = 0.5 ## Specified value for experimental cases
    V = (2/3) * r * Fr_num
#     print("Hour= %d, Fr_num= %.2f, Mean_velocity= %.2f, Ck= %.2f, Rad= %.2f, r= %.2f, V= %.2f"\
#           %(hour, Fr_num, mean_velocity, Ck, Rad, (1 - Rad * ((Wr[Cat_num, hour] - Wr[Cat_num, hour-1]) /\
#                  (Flow_area[Cat_num][hour] - Flow_area[Cat_num][hour-1]))), V))

#     Ck = pow((S0/n), 0.5) * (5/3) * pow(Rad, 2/3) - ((2/3) * pow(Rad, 5/3)) *\
#     (2 * pow((1 + pow(z, 2)) / (Bw + 2 * SH_upstream * z), 0.5)) 
#     v = Ck - mean_velocity # v= relative celerity of kinematic wave velocity   
#     w = pow(g * SH_upstream, 0.5) # w= relative celerity of dynamic wave        
#     V = v / w # V= Vedernikov number (0<V<1)
    
    ## Time-dependent LRMC parameters
    C = Ck * ((SW_tmax / SW_nt) / dx) # C= Courant number, Ck= flood wave celerity, dx= channel length
    q = (Area * mean_velocity) / Bw
    ## D the cell Reynolds number (Ponce, 2001)
    D = (q / (S0 * Ck * dx)) * (1 - pow(V, 2)) # q= unit-width discharge
    C0 = (-1 + C + D) / (1 + C + D)
    C1 = (1 + C - D) / (1 + C + D)
    C2 = (1 - C + D) / (1 + C + D)
    C3 = (2 * C) / (1 + C + D)
#     print("C= %.2f, D= %.2f, V= %.2f, C3= %.2f, S0= %.2f, q= %.2f, 1-pow(V,2)= %.2f" %(C, D, V, C3, S0, q, (1 - pow(V, 2))))

    ## The topmost upstream boundary condition
    str_dis[0, :] = Area * (mean_velocity * (3600)) # str_dis unit: m3/hr, mean_velocity: m/s, Area: m2
    ## Initial condition streamflow 
    str_dis[:, 0] = Area * (mean_velocity * (3600))
    
    for t in range(0, SW_nt):
        for i in range(0, Chan_grid_num):
            str_dis[i+1, t+1] = (C0 * str_dis[i, t+1]) + (C1 * str_dis[i, t]) + (C2 * str_dis[i+1, t])\
            + (C3 * (Qlat)) # Qlat: m3/hr, str_dis: m3/hr
    
    for j in range(Chan_totalLength - Chan_grid_num, Chan_totalLength):
#         r = pow(Bw, 2) + (4 * z * (str_dis[j-(Chan_totalLength - Chan_grid_num), SW_nt] / (mean_velocity * 3600)))
        r = pow(Bw, 2) + (4 * (z * (((Geometry*j/Chan_totalLength)+1))) * (str_dis[j-(Chan_totalLength - Chan_grid_num), SW_nt] / (mean_velocity * 3600)))
        ## SH_profile unit: m
#         SH_profile[j][hour] = (((-(Bw * (1+(j+1)/Chan_totalLength))) + mt.sqrt(r)) / (2 * z))
        SH_profile[j][hour] = (((-Bw) + mt.sqrt(r)) / (2 * (z * (((Geometry*j/Chan_totalLength)+1)))))

        Streamflow_profile[j][hour] = str_dis[j-(Chan_totalLength - Chan_grid_num), SW_nt]
    
    SH_inlet[Cat_num][hour] = SH_profile[Chan_totalLength - Chan_grid_num][hour]
    
    if hour < int(travel_time):
        SH_outlet[Cat_num][hour] = SH_atHW21['River_stage'][1]
    elif hour >= int(travel_time):
        SH_outlet[Cat_num][hour] = SH_profile[Chan_totalLength-1][hour-(int(travel_time))]
    
    return SH_inlet[Cat_num][hour], SH_outlet[Cat_num][hour], SH_profile[(Chan_totalLength - Chan_grid_num):Chan_totalLength, hour]

In [ ]:
### Manning approximation at the upstream node of the corresponding catchment 
def Manning(StreamHeight):
    
#     Area_chan = Bw * StreamHeight + z * pow(StreamHeight, 2)
#     Area_chan = Bw * StreamHeight + (z*(1+1/sum(dis.Chan_grid_num[0:len(dis.Chan_indi)]))) * pow(StreamHeight, 2)
    Area_chan = (2*z + Bw) * StreamHeight

#     Wp = Bw + 2 * StreamHeight * pow((1 + pow(z, 2)), 0.5)
#     Wp = Bw + 2 * StreamHeight * pow((1 + pow((z*(1+1/sum(dis.Chan_grid_num[0:len(dis.Chan_indi)]))), 2)), 0.5)
    Wp = Bw + 2 * pow((pow(StreamHeight, 2) + pow(z, 2)), 0.5)

    R = Area_chan / Wp
    
    ## u is the mean velocity of the normal steady flow
    u = ((1/n) * pow(R, 2/3) * pow(S0, 1/2)) # u: m/s

    return Area_chan, R, u 